In [ ]:
# AIRE Micro Tutor Identity Cell
import datetime as dt
learner_id = input('Enter your learner ID (email or handle): ').strip()
learner_role = input('Enter your learner role (e.g., researcher, educator): ').strip()
resource_id = 'notebook:embeddings_basics'
session_id = f"{learner_id}-{dt.datetime.utcnow().isoformat()}"

In [ ]:
# AIRE Micro Tutor Imports
from prompt_tutor import evaluate_prompt
from aire_telemetry import log_event
from resources_map import RESOURCE_MAP

In [ ]:
import sys, subprocess, os
from pathlib import Path

# Colab Setup
if "google.colab" in sys.modules:
    print("Running in Google Colab. Installing dependencies...")
    subprocess.run(["pip", "install", "-q", "pandas", "numpy", "scikit-learn", "requests", "pydantic", "jsonschema", "plotly", "tqdm"])
    
    # Check for data
    if not (Path.cwd() / "data").exists():
        print("Data directory not found. Cloning repository...")
        subprocess.run(["git", "clone", "https://github.com/aire-program/aire-researcher-sandbox.git", "_repo"])
        
        # Move data and scripts to current directory
        if (Path("_repo/data").exists()):
            print("Moving data and scripts...")
            subprocess.run(["mv", "_repo/data", "."])
            subprocess.run(["mv", "_repo/scripts", "."])
            subprocess.run(["rm", "-rf", "_repo"])
        else:
            print("Warning: Data not found in cloned repo.")
    else:
        print("Data directory found.")


# Embeddings API Example

**What**: Generate and use text embeddings via a simulated API.

**Why**: Many modern AI workflows offload heavy computation (like embedding generation) to specialized APIs.

**How**:
1. **Send text** to the embedding endpoint.
2. **Receive vectors** in response.
3. **Use vectors** for similarity calculations.

**Key Concept**: **Embeddings** are dense numerical representations of text where similar meanings are close together in vector space.

By the end of this notebook, you will have completed the listed steps and produced the outputs described in the success criteria.

### Success criteria
- You generated embeddings for sample texts.
- You computed similarity and inspected scores.
- You executed a simple search over the texts.

In [ ]:
import sys
from pathlib import Path

repo_root = Path.cwd()
for candidate in [repo_root, repo_root.parent, repo_root.parent.parent]:
    if (candidate / "api" / "python" / "client_embeddings.py").exists():
        sys.path.append(str(candidate))
        break

from api.python.client_embeddings import EmbeddingsClient

texts = [
    "Synthetic research abstract about reproducibility.",
    "Notes on experimental design and treatment arms.",
    "Overview of responsible AI documentation practices.",
]

client = EmbeddingsClient(max_features=32)
embeddings = client.embed(texts)
embeddings.shape


## Pairwise similarity

In [ ]:
similarity = client.similarity(texts)
similarity


### If you get stuck / What to try next

If you get stuck: rerun installs and ensure sample texts are defined. What to try next: connect embeddings to retrieval workflows or try the text notebooks for qualitative checks.

In [ ]:
# AIRE Micro Tutor Prompt + Feedback Cell
from aire_llm_client import chat_completion

def call_llm(prompt_text: str) -> str:
    messages = [
        {'role': 'system', 'content': 'You are an AI helper responding concisely.'},
        {'role': 'user', 'content': prompt_text},
    ]
    return chat_completion(messages)

prompt_text = input('Enter the prompt you want to test: ').strip()
ai_response = call_llm(prompt_text)
evaluation = evaluate_prompt(prompt_text, learner_role)

clarity = evaluation.get('clarity_score')
context = evaluation.get('context_score')
constraints = evaluation.get('constraints_score')
evaluation_score = evaluation.get('evaluation_score')
primary_weakness = evaluation.get('primary_weakness', '')
recommended_resource_id = RESOURCE_MAP.get(primary_weakness) or RESOURCE_MAP.get('evaluation')

log_event(
    event_name='micro_tutor_evaluation',
    user_id=learner_id or session_id,
    metadata={
        'resource_id': resource_id,
        'learner_role': learner_role,
        'clarity': clarity,
        'context': context,
        'constraints': constraints,
        'evaluation_score': evaluation_score,
        'primary_weakness': primary_weakness,
    },
)

print('\n--- AI Response ---')
print(ai_response)
print('\n--- Feedback ---')
print(evaluation.get('summary', 'No summary provided.'))
print('Primary weakness:', primary_weakness or 'n/a')
print('Strengths:', ', '.join(evaluation.get('strengths', [])) or 'n/a')
print('Suggestions:', ', '.join(evaluation.get('suggestions', [])) or 'n/a')
print('\nRecommended resource:', recommended_resource_id or 'n/a')